<a href="https://colab.research.google.com/github/sairaawahid/Employee-Attrition-Predictor-for-HR-Analytics/blob/main/employee_attrition_predictor_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# app.py

import streamlit as st
import pandas as pd
import numpy as np
import joblib
import shap
import json
import matplotlib.pyplot as plt

# Load model and expected input features
model = joblib.load("models/xgboost_optimized_model.pkl")
with open("models/model_features.json", "r") as f:
    model_features = json.load(f)

explainer = shap.TreeExplainer(model)

# App settings
st.set_page_config(page_title="Employee Attrition Predictor", layout="centered")
st.title("🧠 Employee Attrition Predictor")
st.markdown("Predict whether an employee is likely to leave based on workplace features. Built with XGBoost and SHAP.")

# Initialize session state for prediction history
if "history" not in st.session_state:
    st.session_state["history"] = []

# Sample data toggle
use_sample = st.checkbox("🔁 Use Sample Employee Data")

# Sidebar inputs
st.sidebar.header("📋 Enter Employee Information")

def user_input():
    if use_sample:
        Age = 28
        MonthlyIncome = 4800
        OverTime_Yes = "Yes"
        JobSatisfaction = 2
        EnvironmentSatisfaction = 2
        DistanceFromHome = 20
        BusinessTravel_Travel_Frequently = "Yes"
        NumCompaniesWorked = 4
        StockOptionLevel = 0
        YearsWithCurrManager = 1
        WorkLifeBalance = 2
    else:
        Age = st.sidebar.slider("Age", 18, 60, 35)
        MonthlyIncome = st.sidebar.slider("Monthly Income", 1000, 20000, 5000)
        OverTime_Yes = st.sidebar.selectbox("Works Overtime?", ["Yes", "No"])
        JobSatisfaction = st.sidebar.slider("Job Satisfaction (1-4)", 1, 4, 3)
        EnvironmentSatisfaction = st.sidebar.slider("Environment Satisfaction (1-4)", 1, 4, 3)
        DistanceFromHome = st.sidebar.slider("Distance From Home (km)", 1, 50, 10)
        BusinessTravel_Travel_Frequently = st.sidebar.selectbox("Travels Frequently?", ["Yes", "No"])
        NumCompaniesWorked = st.sidebar.slider("Number of Companies Worked", 0, 10, 2)
        StockOptionLevel = st.sidebar.slider("Stock Option Level (0-3)", 0, 3, 1)
        YearsWithCurrManager = st.sidebar.slider("Years with Current Manager", 0, 20, 4)
        WorkLifeBalance = st.sidebar.slider("Work-Life Balance (1-4)", 1, 4, 3)

    if st.sidebar.button("🔄 Reset Form"):
        st.experimental_rerun()

    return pd.DataFrame([{
        'Age': Age,
        'MonthlyIncome': MonthlyIncome,
        'OverTime_Yes': 1 if OverTime_Yes == "Yes" else 0,
        'JobSatisfaction': JobSatisfaction,
        'EnvironmentSatisfaction': EnvironmentSatisfaction,
        'DistanceFromHome': DistanceFromHome,
        'BusinessTravel_Travel_Frequently': 1 if BusinessTravel_Travel_Frequently == "Yes" else 0,
        'NumCompaniesWorked': NumCompaniesWorked,
        'StockOptionLevel': StockOptionLevel,
        'YearsWithCurrManager': YearsWithCurrManager,
        'WorkLifeBalance': WorkLifeBalance
    }])

# Collect input
input_df = user_input()

# Reorder input columns to match model training
input_df = input_df.reindex(columns=model_features)

# Run prediction
prediction = model.predict(input_df)[0]
probability = model.predict_proba(input_df)[0][1]

# Display prediction results
st.subheader("🎯 Prediction Result")
st.write("Attrition Prediction:", "🔴 Yes" if prediction == 1 else "🟢 No")
st.write("Probability of Leaving:", f"{probability:.2%}")

# Toggle SHAP
show_shap = st.checkbox("📊 Show SHAP Explanation")

if show_shap:
    shap_values = explainer.shap_values(input_df)

    # Force plot
    st.subheader("🔍 SHAP Force Plot (Local Explanation)")
    plt.clf()
    shap.force_plot(explainer.expected_value, shap_values[0], input_df.iloc[0], matplotlib=True, show=False)
    fig = plt.gcf()
    st.pyplot(fig)

    # Bar plot
    st.subheader("📈 SHAP Feature Importance (Bar Plot)")
    plt.clf()
    shap.summary_plot(shap_values, input_df, plot_type="bar", show=False)
    fig = plt.gcf()
    st.pyplot(fig)

    # Summary (beeswarm)
    st.subheader("🌈 SHAP Summary (Beeswarm Plot)")
    plt.clf()
    shap.summary_plot(shap_values, input_df, show=False)
    fig = plt.gcf()
    st.pyplot(fig)

# Store to prediction history
if st.button("💾 Add to Prediction History"):
    record = {
        "Age": input_df.iloc[0]["Age"],
        "MonthlyIncome": input_df.iloc[0]["MonthlyIncome"],
        "OverTime": "Yes" if input_df.iloc[0]["OverTime_Yes"] == 1 else "No",
        "Attrition Risk": f"{probability:.2%}",
        "Prediction": "Yes" if prediction == 1 else "No"
    }
    st.session_state["history"].append(record)
    st.success("Prediction saved to history.")

# Show history
if st.session_state["history"]:
    st.subheader("📚 Prediction History")
    st.dataframe(pd.DataFrame(st.session_state["history"]))
